# TUM KATEGORILERIN MERGE EDILMESI -FINAL CVS 

### Imports Libraries

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import cufflinks as cf
%matplotlib inline 

from scipy import stats


pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings("ignore")

### Import Local `utils1` Modul

In [3]:
#  FONKSIYONLARIMIZI DERLEDIGIMIZ `UTILS` ADINDAKI MODULU BU NOTEBOOK ICINE IMPORT EDIYORUZ

import sys

# utils1.py dosyasının bulunduğu dizini ham dize olarak ekleyin
sys.path.append(r"C:\Users\duygu\Desktop\00-GitHub-Repo\00-1INTERNSHIP\10_UTILITIES")

# 'utils1.py' dosyasında olusturulan Utils sınıfını import et
from utils1 import Utils, UtilsAnalysis

# utils1 icindeki tum class'lardan bir nesne oluştur
utils = Utils()
utils_analysis = UtilsAnalysis()

In [3]:
# utils nesnesindeki tüm fonksiyonları ve öznitelikleri listelemek için
dir(utils)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'df',
 'read_excel_page',
 'save_and_load_csv']

In [4]:
# utils nesnesindeki tüm fonksiyonları ve öznitelikleri listelemek için
dir(UtilsAnalysis)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'detect_anomalies',
 'duplicate_values',
 'find_non_numeric_values',
 'get_value_count',
 'missing_values',
 'na_ratio_plot',
 'numeric_summary',
 'object_summary',
 'rename_columns_by_position',
 'set_df']

### User Defined Funcs for Summary of Data

In [4]:
# =============== User-Defined-Function ==========================

def rename_columns_by_position(df, new_column_names):
    """
    DataFrame'deki sütun adlarını pozisyona göre yeniden adlandırır.

    Parameters:
        df (pd.DataFrame): Sütun adlarını değiştirmek istediğiniz DataFrame.
        new_column_names (list): Yeni sütun adlarının listesi.

    Returns:
        pd.DataFrame: Yeni sütun adlarıyla güncellenmiş DataFrame.
    """
    # Sütun sayısının uyumlu olup olmadığını kontrol et
    if len(new_column_names) != len(df.columns):
        raise ValueError("Sütun sayısı ile yeni adların sayısı eşleşmiyor.")
    
    # Yeni sütun adlarını atama
    df.columns = new_column_names   
    return df


#================ Categorical Features Summary ===================
def object_summary(df):
    obs = df.shape[0]
    duplicate_count = df.duplicated().sum()

    # Kategorik sütunlar için özetleme
    object_df = df.select_dtypes(include='object')
    
    # Yeni bir boş DataFrame oluşturma
    summary_df = pd.DataFrame(index=object_df.columns)

    summary_df['Dtype'] = object_df.dtypes
    summary_df['Counts'] = object_df.count()
    summary_df['Nulls'] = object_df.isnull().sum()
    summary_df['NullPercent'] = (object_df.isnull().sum() / obs) * 100
    summary_df['Top'] = object_df.apply(lambda x: x.mode().iloc[0] if not x.mode().empty else '-')
    summary_df['Frequency'] = object_df.apply(lambda x: x.value_counts().max() if not x.value_counts().empty else '-')
    summary_df['Uniques'] = object_df.nunique()

    # UniqueValues sütununu kontrol ederek ekleme (dize olarak)
    summary_df['UniqueValues'] = object_df.apply(
        lambda x: ', '.join(map(str, x.unique()[:10])) + '...' if x.nunique() > 10 else ', '.join(map(str, x.unique()))
    )

    # DataFrame şekli ve tekrar eden satır sayısını ekrana yazdırma
    print(f'1. Data shape (rows, columns): {df.shape}')
    print(f'2. Number of duplicate rows: {duplicate_count}')
    return summary_df


#================ Numerical Features Summary ===================
def numeric_summary(df):
    obs = df.shape[0]
    duplicate_count = df.duplicated().sum()

    # Numerik sütunlar için özetleme
    numeric_df = df.select_dtypes(include=['float64', 'int64'])

    # Yeni bir boş DataFrame oluşturma
    summary_df = pd.DataFrame(index=numeric_df.columns)

    summary_df['Dtype'] = numeric_df.dtypes
    summary_df['Counts'] = numeric_df.count()
    summary_df['Nulls'] = numeric_df.isnull().sum()
    summary_df['NullPercent'] = (numeric_df.isnull().sum() / obs) * 100
    summary_df['Mean'] = numeric_df.mean()
    summary_df['Std'] = numeric_df.std()
    summary_df['Min'] = numeric_df.min()
    summary_df['25%'] = numeric_df.quantile(0.25)
    summary_df['50% (Median)'] = numeric_df.median()
    summary_df['75%'] = numeric_df.quantile(0.75)
    summary_df['Max'] = numeric_df.max()

    # DataFrame şekli ve tekrar eden satır sayısını ekrana yazdırma
    print(f'1. Data shape (rows, columns): {df.shape}')
    print(f'2. Number of duplicate rows: {duplicate_count}')
    return summary_df
    

#========== Get count and percentage of values for each column =================
def get_value_count(df, column_name):
    """
    This function calculates and returns a DataFrame with the value counts and 
    their corresponding percentages for a specified column in the DataFrame.
    """
    
    vc = df[column_name].value_counts()
    vc_norm = df[column_name].value_counts(normalize=True)
    
    vc = vc.rename_axis(column_name).reset_index(name='counts')
    vc_norm = vc_norm.rename_axis(column_name).reset_index(name='percent')
    vc_norm['percent'] = (vc_norm['percent'] * 100).map('{:.2f}%'.format)
    
    df_result = pd.concat([vc[column_name], vc['counts'], vc_norm['percent']], axis=1)
    return df_result


#============== Checks duplicates and drops them ==========================

def duplicate_values(df):
    print("Duplicate check...")
    num_duplicates = df.duplicated(subset=None, keep='first').sum()
    if num_duplicates > 0:
        print("There are", num_duplicates, "duplicated observations in the dataset.")
        df.drop_duplicates(keep='first', inplace=True)
        print(num_duplicates, "duplicates were dropped!")
        print("No more duplicate rows!")
    else:
        print("There are no duplicated observations in the dataset.")



# ========== User-Defined-Function for Missing Values ============
def missing_values(df):
    """This function calculates the missing values count and their percentage in a DataFrame."""

    missing_count = df.isnull().sum()
    value_count = df.isnull().count()
    missing_percentage = round(missing_count / value_count * 100, 2)
    
    # Format the percentage as '0.00%' with % symbol
    missing_percentage_formatted = missing_percentage.map("{:.2f}%".format)
    # Create a DataFrame to store the results
    missing_df = pd.DataFrame({"count": missing_count, "percentage": missing_percentage_formatted}) 
    return missing_df


# ========== Plotting Missing Values  ===========================
def na_ratio_plot(df):
    """Plots the ratio of missing values for each feature and prints the count of missing values."""
    
    sns.displot(df.isna().melt(value_name='Missing_data',var_name='Features')\
                ,y='Features',hue='Missing_data',multiple='fill',aspect=9/8)

    print(df.isna().sum()[df.isna().sum()>0])

    
    
#========== Detecting Anomalies ================================

def detect_anomalies(df, column_name):
    """
    Detects values with unusual (non-alphanumeric) characters in a column.
    Returns: list: Detected unusual character values.
    """
    # Get the unique values in the column
    unique_values = df[column_name].unique()    
    # Detect values with unusual characters (non-alphanumeric)
    unusual_characters = [val for val in unique_values if isinstance(val, str) and not val.isalnum()]
    
    # Return the list as a single string with values separated by commas
    return ', '.join(unusual_characters)


#========== Detecting Non-Numerical Characters ===========================

import re

def find_non_numeric_values(df, column_name):
    """
    Finds unique non-numeric values in a specified column of the DataFrame.
    """
    pattern = r'\D+'  # Pattern to match non-numeric characters
    # Find and flatten non-numeric values, then ensure uniqueness with set
    return set(re.findall(pattern, ' '.join(df[column_name].astype(str))))


#===============Plot all Categoricals Bar Chart ===========================
def plot_categorical_bar_graphs(df):
    """
    DataFrame'deki tüm kategorik veriler için bar grafikleri oluşturur.
    
    Args:
    - df (DataFrame): Kategorik verileri içeren DataFrame.
    """
    # Kategorik sütunları seçme
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns

    # Her kategorik sütun için bar grafiği oluşturma
    for column in categorical_columns:
        plt.figure(figsize=(15, 5))
        ax = sns.countplot(
            x=column, 
            data=df, 
            order=df[column].value_counts().index, 
            palette='Set2'
        )
        ax.bar_label(ax.containers[0], rotation=45)
        plt.xlabel(column, fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.title(f'Bar Graph of {column}', fontsize=20)
        plt.xticks(rotation=45, ha='right', fontsize=12)
        plt.show()
        
#======================================================================

def combine_dataframes(df_list):
    """
    Verilen DataFrame listesini alt alta birleştirir.
    
    :param df_list: Birleştirilecek DataFrame'lerin listesi
    :return: Birleştirilmiş DataFrame
    """
    # DataFrame'leri alt alta birleştir
    combined_df = pd.concat(df_list, ignore_index=True, sort=False)
    
    return combined_df

#====================================================================

def move_column(df, column_name, new_index):
    """
    Verilen bir sütun adını belirtilen yeni indekse taşır.
    
    :param df: DataFrame
    :param column_name: Taşınacak sütunun adı
    :param new_index: Yeni indeks numarası
    :return: Sütunları yeniden düzenlenmiş DataFrame
    """
    # Mevcut sütunları listeye al
    columns = list(df.columns)
    
    # Taşınacak sütunu kaldır
    columns.remove(column_name)
    
    # Sütunu istenilen yeni indekse ekle
    columns.insert(new_index, column_name)
    
    # DataFrame'i yeni sütun sırasına göre düzenle
    return df[columns]


## Ameliyat Masasi

In [ ]:
df_ameliyat 
df_otoklav
df_hidrojen_peroktif
df_kartus
df_reverse
df_yikama
df_oksijen

In [140]:
import pandas as pd

# Dosya yolunu belirtelim
file_path = r"C:\Users\duygu\Desktop\00-GitHub-Repo\00-1INTERNSHIP\00_Data\01_Satis_Uretim\02_Birlestirilmis_Kategoriler\final_kategori_csv\combined_ameliyat.csv"

# CSV dosyasını okuyalım
try:
    df_ameliyat = pd.read_csv(file_path)
    # İlk birkaç satırı göstereceğiz
    df_ameliyat.head()
except FileNotFoundError:
    "Dosya yolunda bir hata olabilir veya dosya mevcut olmayabilir."
    
df_ameliyat.head(2)


,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi
0,AMELIYAT MASASI,ÖZEL,2018/006,2018-06-29,ÖZEL FSM TIP MERKEZİ,ÖZEL FSM TIP MERKEZİ,İSTANBUL,GOLDBERG 5S,AMELİYAT MASASI,*,2018/02-012,20 GÜN,2018-07-18,NaN,NaN
1,AMELIYAT MASASI,DEMO,2018/007,2018-02-14,DEMO,DERİNCE EĞİTİM VE ARAŞTIRMA HASTANESİ,KOCAELİ,GOLDBERG 5S,AMELİYAT MASASI,*,2018/02-003,21 GÜN,NaN,NaN,NaN


### Kategori degerleri buyuk harfle duzenlendi

In [29]:
# Kategori drop et
#df_ameliyat = df_ameliyat.drop('kategori', axis=1)

In [30]:
# # 'kategori' sütununu birinci indekste degerleri buyuk harfle duzenle

# df_ameliyat['kategori'] = 'AMELIYAT MASASI'

# columns = ['kategori'] + list(df_ameliyat.columns[:-1])
# df_ameliyat = df_ameliyat[columns]

# df_ameliyat.head(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi
0,AMELIYAT MASASI,ÖZEL,2018/006,2018-06-29,ÖZEL FSM TIP MERKEZİ,ÖZEL FSM TIP MERKEZİ,İSTANBUL,GOLDBERG 5S,AMELİYAT MASASI,*,2018/02-012,20 GÜN,2018-07-18,NaN,NaN
1,AMELIYAT MASASI,DEMO,2018/007,2018-02-14,DEMO,DERİNCE EĞİTİM VE ARAŞTIRMA HASTANESİ,KOCAELİ,GOLDBERG 5S,AMELİYAT MASASI,*,2018/02-003,21 GÜN,NaN,NaN,NaN


In [6]:
df_ameliyat.columns

Index(['kategori', 'satis_turu', 'siparis_no', 'siparis_tarihi', 'siparisi_veren', 'kurulum_yeri', 'il', 'urun_modeli', 'urun_adi', 'litre', 'seri_no', 'teslim_suresi', 'teslim_tarihi', 'siparis_durumu', 'irsaliye_tarihi'], dtype='object')

## Otoklav

In [141]:
import pandas as pd

# Dosya yolunu belirtelim
file_path = r"C:\Users\duygu\Desktop\00-GitHub-Repo\00-1INTERNSHIP\00_Data\01_Satis_Uretim\02_Birlestirilmis_Kategoriler\final_kategori_csv\combined_otoklav.csv"

# CSV dosyasını okuyalım
try:
    df_otoklav = pd.read_csv(file_path)
    # İlk birkaç satırı göstereceğiz
    df_otoklav.head()
except FileNotFoundError:
    "Dosya yolunda bir hata olabilir veya dosya mevcut olmayabilir."
    
df_otoklav.head(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,irsaliye_tarihi
0,OTOKLAV,ÖZEL,2018/001,2018-01-04,YENİ HAYAT TIP MERKEZİ-ORJİN SAĞLIK HİZM.,YENİ HAYAT TIP MERKEZİ-ORJİN SAĞLIK HİZM.,İSTANBUL,STR 6610 D,422 LT ÇİFT KAPILI OTOKLAV CİHAZI,422 LİTRE,2018/01-001,30,NaN,NaN
1,OTOKLAV,DMO,2018/072,2017-12-28,DMO GENEL MÜDÜRLÜK,T.C. GIDA TARIM VE HAYV. BAKANLIĞI GIDA VE KON...,ANKARA,7717 S,2000 LT ÇİFT KAPILI LABARATUVAR TİP BUHAR STER...,2000 LİTRE,2018/01-002,100 GÜN,2018-06-22,NaN


In [30]:
df_otoklav.columns

Index(['kategori', 'satis_turu', 'siparis_no', 'siparis_tarihi', 'siparisi_veren', 'kurulum_yeri', 'il', 'urun_modeli', 'urun_adi', 'litre', 'seri_no', 'teslim_suresi', 'teslim_tarihi', 'irsaliye_tarihi'], dtype='object')

## Hidrojen Peroksit

In [142]:
import pandas as pd

# Dosya yolunu belirtelim
file_path = r"C:\Users\duygu\Desktop\00-GitHub-Repo\00-1INTERNSHIP\00_Data\01_Satis_Uretim\02_Birlestirilmis_Kategoriler\final_kategori_csv\combined_hidrojen.csv"

# CSV dosyasını okuyalım
try:
    df_hidrojen_peroktif = pd.read_csv(file_path)
    # İlk birkaç satırı göstereceğiz
    df_hidrojen_peroktif.head()
except FileNotFoundError:
    "Dosya yolunda bir hata olabilir veya dosya mevcut olmayabilir."
    
df_hidrojen_peroktif.head(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi
0,HIDROJEN PEROKSIT,YURTDIŞI,2018/036,2018-01-23 00:00:00,MERİVAN,MERİVAN,KUZEY IRAK,GP 120,120 LT PLAZMA CİHAZI,120 LT,2018/04-001,49 GÜN,2018-01-27,NaN,NaN
1,HIDROJEN PEROKSIT,DMO,2018/037,DMO,DMO MERSİN BÖLGE MÜDÜRLÜĞÜ,BOR DEVLET HASTANESİ,NİĞDE,GP 120,120 LT PLAZMA CİHAZI,120 LT,2018/04-002,50 GÜN,2018-05-14,NaN,NaN


### Kategori degerleri buyuk harfle duzenlendi

In [21]:
# # Kategori drop et
# df_hidrojen_peroktif = df_hidrojen_peroktif.drop('kategori', axis=1)

In [22]:
# # 'kategori' sütununu birinci indekste degerleri buyuk harfle duzenle

# df_hidrojen_peroktif['kategori'] = 'HIDROJEN PEROKSIT'

# columns = ['kategori'] + list(df_hidrojen_peroktif.columns[:-1])
# df_hidrojen_peroktif = df_hidrojen_peroktif[columns]

# df_hidrojen_peroktif.head(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi
0,HIDROJEN PEROKSIT,YURTDIŞI,2018/036,2018-01-23 00:00:00,MERİVAN,MERİVAN,KUZEY IRAK,GP 120,120 LT PLAZMA CİHAZI,120 LT,2018/04-001,49 GÜN,2018-01-27,NaN,NaN
1,HIDROJEN PEROKSIT,DMO,2018/037,DMO,DMO MERSİN BÖLGE MÜDÜRLÜĞÜ,BOR DEVLET HASTANESİ,NİĞDE,GP 120,120 LT PLAZMA CİHAZI,120 LT,2018/04-002,50 GÜN,2018-05-14,NaN,NaN


In [10]:
df_hidrojen_peroktif.columns

Index(['kategori', 'satis_turu', 'siparis_no', 'siparis_tarihi', 'siparisi_veren', 'kurulum_yeri', 'il', 'urun_modeli', 'urun_adi', 'litre', 'seri_no', 'teslim_suresi', 'teslim_tarihi', 'siparis_durumu', 'irsaliye_tarihi'], dtype='object')

### Rename Columns

In [20]:
new_column_names = [
    'kategori',
    'satis_turu', 
    'siparis_no', 
    'siparis_tarihi',
    'siparisi_veren',
    'kurulum_yeri',
    'il',
    'urun_modeli',
    'urun_adi',
    'litre',
    'seri_no',
    'teslim_suresi',
    'teslim_tarihi',
    'siparis_durumu',  # siparisin_tarihi isim degisti
    'irsaliye_tarihi',
    #'uts_no'
]

df_hidrojen_peroktif = rename_columns_by_position(df_hidrojen_peroktif, new_column_names)

df_hidrojen_peroktif.head(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi
0,Hidrojen Peroksit,YURTDIŞI,2018/036,2018-01-23 00:00:00,MERİVAN,MERİVAN,KUZEY IRAK,GP 120,120 LT PLAZMA CİHAZI,120 LT,2018/04-001,49 GÜN,2018-01-27,NaN,NaN
1,Hidrojen Peroksit,DMO,2018/037,DMO,DMO MERSİN BÖLGE MÜDÜRLÜĞÜ,BOR DEVLET HASTANESİ,NİĞDE,GP 120,120 LT PLAZMA CİHAZI,120 LT,2018/04-002,50 GÜN,2018-05-14,NaN,NaN


## Kartus

In [143]:
import pandas as pd

# Dosya yolunu belirtelim
file_path = r"C:\Users\duygu\Desktop\00-GitHub-Repo\00-1INTERNSHIP\00_Data\01_Satis_Uretim\02_Birlestirilmis_Kategoriler\combined_kartus.csv"

# CSV dosyasını okuyalım
try:
    df_kartus = pd.read_csv(file_path)
    # İlk birkaç satırı göstereceğiz
    df_kartus.head()
except FileNotFoundError:
    "Dosya yolunda bir hata olabilir veya dosya mevcut olmayabilir."
    
df_kartus.sample(4)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,aciklama,solusyon_fatura,bidon_acilisi
21938,KARTUS,ÖZEL,2023/134,2023-11-14 00:00:00,ÖZEL MEDİKALPARK MALTEPE HASTANESİ (SAMSUN MED...,NaN,İSTANBUL,NaN,KARTUŞ,NaN,2023/07-4814,NaN,NaN,NaN,NaN,NaN
3564,KARTUS,SATIŞ,2019/099,2019-10-16,SURİYE GÖREV GÜCÜ,NaN,SURİYE,NaN,KARTUŞ,NaN,2019/07-1749,NaN,NaN,SARI KARTUŞ,25.4.2019-HPS2019-00387,2019-10-08
16526,KARTUS,DOĞRUDAN TEMİN,NaN,2022-11-30,KİLİS ALAATTİN YAVAŞÇA DEVLET HASTANESİ,NaN,KİLİS,NaN,KARTUŞ,NaN,2022/07-4840,NaN,NaN,NaN,NaN,NaN
10470,KARTUS,SATIŞ,2021/105,2021-09-23,UKRAYNA,NaN,UKRAYNA,NaN,KARTUŞ,NaN,2021/07-3301,NaN,NaN,SATIŞ,NaN,NaN


In [144]:
print(df_kartus.shape)
df_kartus.columns

(30126, 16)


Index(['kategori', 'satis_turu', 'siparis_no', 'siparis_tarihi', 'siparisi_veren', 'kurulum_yeri', 'il', 'urun_modeli', 'urun_adi', 'litre', 'seri_no', 'teslim_suresi', 'teslim_tarihi', 'aciklama', 'solusyon_fatura', 'bidon_acilisi'], dtype='object')

## Reverse Osmos

In [155]:
import pandas as pd

# Dosya yolu belirtelim
file_path = r"C:\Users\duygu\Desktop\00-GitHub-Repo\00-1INTERNSHIP\00_Data\01_Satis_Uretim\02_Birlestirilmis_Kategoriler\combined_reverse.csv"

# CSV dosyasını doğru ayırıcı ile okuyalım
try:
    # Veriyi okurken ayırıcı olarak noktalı virgül (;) kullanalım
    df_reverse = pd.read_csv(file_path)
    # İlk birkaç satırı gösterelim
    df_reverse.head()
except FileNotFoundError:
    print("Dosya yolunda bir hata olabilir veya dosya mevcut olmayabilir.")

df_reverse.sample(5)


,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi
11,REVERSE OSMOS,İHALE,2018/349,1.12.2018,ALANYA ALAADDİN KEYKUBAT ÜNİVERSİTESİ,ALANYA ALAADDİN KEYKUBAT ÜNİVERSİTESİ,ALANYA,PREMİUM 440,NaN,1000 LT,2018/08-012,20 GÜN,NaN
122,REVERSE OSMOS,YURTDIŞI,2022/318,10.08.2022,IRAK/TKMS,IRAK,IRAK,PREMIUM 640,NaN,1500 LT/SAAT,2022/08-033,21,31.08.2022
198,REVERSE OSMOS,YURTDIŞI,2024/249,20.08.2024,IRAK,IRAK,IRAK,ULTIMATE 280,2000 LT/SAAT REVERSE OSMOS SU ARITMA SİSTEMİ,2000 LT/SAAT,2024/08-028,20,9.09.2024
161,REVERSE OSMOS,İHALE,2023/239,6.07.2023,KAMU HASTANELERİ GENEL MÜDÜRLÜĞÜ,ANKARA 75. YIL AĞIZ VE DİŞ SAĞLIĞI BÖLGESİ,ANKARA,PREMIUM 340,750 LT/SAAT REVERSE OSMOS SU ARTIMA SİSTEMİ,750 LT/SAAT,2023/08-028,20,26.07.2023
80,REVERSE OSMOS,ÖZEL,2021/366,12.10.2021,ORDEM YAPI İTH.İHR.TİC.LTD.ŞTİ.,TRAKYA ÜNİVERSİTESİ,EDİRNE,PREMİUM 640,NaN,1800 lt/saat,2021/08-016,30,11.11.2021


In [156]:
print(df_reverse.shape)
df_reverse.columns

(204, 13)


Index(['kategori', 'satis_turu', 'siparis_no', 'siparis_tarihi', 'siparisi_veren', 'kurulum_yeri', 'il', 'urun_modeli', 'urun_adi', 'litre', 'seri_no', 'teslim_suresi', 'teslim_tarihi'], dtype='object')

## Yikama

In [151]:
import pandas as pd

# Dosya yolunu belirtelim
file_path = r"C:\Users\duygu\Desktop\00-GitHub-Repo\00-1INTERNSHIP\00_Data\01_Satis_Uretim\02_Birlestirilmis_Kategoriler\combined_yikama.csv"

# CSV dosyasını okuyalım
try:
    df_yikama = pd.read_csv(file_path)
    # İlk birkaç satırı göstereceğiz
    df_yikama.head()
except FileNotFoundError:
    "Dosya yolunda bir hata olabilir veya dosya mevcut olmayabilir."
    
df_yikama.head(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi
0,YIKAMA,DMO,2018/041,2018-01-02,DMO İSTANBUL BÖLGE MÜDÜRLÜĞÜ,HUDUT VE SAHİLLER SAĞLIK MÜDÜRLÜĞÜ DESTEK MÜDÜ...,İSTANBUL,TSY 360,360 LT YIKAMA CİHAZI,360 LT,2018/05-005,28 GÜN,NaN,NaN,NaN
1,YIKAMA,DMO,2018/042,2018-01-02,DMO İSTANBUL BÖLGE MÜDÜRLÜĞÜ,HUDUT VE SAHİLLER SAĞLIK MÜDÜRLÜĞÜ DESTEK MÜDÜ...,İSTANBUL,TSY 360,360 LT YIKAMA CİHAZI,360 LT,2018/05-003,29 GÜN,NaN,NaN,NaN


### Kategori degerleri buyuk harfle duzenlendi

In [10]:
# # Kategori drop et
# df_yikama = df_yikama.drop('kategori', axis=1)

In [11]:
# # 'kategori' sütununu birinci indekste degerleri buyuk harfle duzenle
# df_yikama['kategori'] = 'YIKAMA'
# columns = ['kategori'] + list(df_yikama.columns[:-1])
# df_yikama = df_yikama[columns]

In [12]:
df_yikama.head(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparisin_durumu,irsaliye_tarihi
0,YIKAMA,DMO,2018/041,2018-01-02,DMO İSTANBUL BÖLGE MÜDÜRLÜĞÜ,HUDUT VE SAHİLLER SAĞLIK MÜDÜRLÜĞÜ DESTEK MÜDÜ...,İSTANBUL,TSY 360,360 LT YIKAMA CİHAZI,360 LT,2018/05-005,28 GÜN,NaN,NaN,NaN
1,YIKAMA,DMO,2018/042,2018-01-02,DMO İSTANBUL BÖLGE MÜDÜRLÜĞÜ,HUDUT VE SAHİLLER SAĞLIK MÜDÜRLÜĞÜ DESTEK MÜDÜ...,İSTANBUL,TSY 360,360 LT YIKAMA CİHAZI,360 LT,2018/05-003,29 GÜN,NaN,NaN,NaN


### Rename Columns

In [13]:
df_yikama.columns

Index(['kategori', 'satis_turu', 'siparis_no', 'siparis_tarihi', 'siparisi_veren', 'kurulum_yeri', 'il', 'urun_modeli', 'urun_adi', 'litre', 'seri_no', 'teslim_suresi', 'teslim_tarihi', 'siparisin_durumu', 'irsaliye_tarihi'], dtype='object')

In [16]:
new_column_names = [
    'kategori',
    'satis_turu', 
    'siparis_no', 
    'siparis_tarihi',
    'siparisi_veren',
    'kurulum_yeri',
    'il',
    'urun_modeli',
    'urun_adi',
    'litre',
    'seri_no',
    'teslim_suresi',
    'teslim_tarihi',
    'siparis_durumu',  # siparisin_tarihi isim degisti
    'irsaliye_tarihi',
    #'uts_no'
]

df_yikama = rename_columns_by_position(df_yikama, new_column_names)

df_yikama.head(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi
0,YIKAMA,DMO,2018/041,2018-01-02,DMO İSTANBUL BÖLGE MÜDÜRLÜĞÜ,HUDUT VE SAHİLLER SAĞLIK MÜDÜRLÜĞÜ DESTEK MÜDÜ...,İSTANBUL,TSY 360,360 LT YIKAMA CİHAZI,360 LT,2018/05-005,28 GÜN,NaN,NaN,NaN
1,YIKAMA,DMO,2018/042,2018-01-02,DMO İSTANBUL BÖLGE MÜDÜRLÜĞÜ,HUDUT VE SAHİLLER SAĞLIK MÜDÜRLÜĞÜ DESTEK MÜDÜ...,İSTANBUL,TSY 360,360 LT YIKAMA CİHAZI,360 LT,2018/05-003,29 GÜN,NaN,NaN,NaN


## Oksijen

In [153]:
import pandas as pd

# Dosya yolunu belirtelim
file_path = r"C:\Users\duygu\Desktop\00-GitHub-Repo\00-1INTERNSHIP\00_Data\01_Satis_Uretim\02_Birlestirilmis_Kategoriler\combined_oksijen.csv"

# CSV dosyasını okuyalım
try:
    df_oksijen = pd.read_csv(file_path)
    # İlk birkaç satırı göstereceğiz
    df_oksijen.head()
except FileNotFoundError:
    "Dosya yolunda bir hata olabilir veya dosya mevcut olmayabilir."
    
df_oksijen.head(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi,uts_no
0,OKSIJEN,YURTDIŞI,2018/040,5/2/2018,SIMAD UNİVERSİTESİ,UĞUR SAĞLIK HİZMETLERİ A.Ş.,SOMALİ,OXYGOLD-22,NaN,220 LT,2018/06-005,30,6/10/2018,NaN,NaN,NaN
1,OKSIJEN,YURTDIŞI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
print(df_oksijen.shape)

(71, 16)


# **Tüm dosyaları kayıt etme** 

In [31]:
import os

# DataFrame isimlerini ve dosya isimlerini bir araya getirme
dfs = [
    (df_ameliyat, "combined_ameliyat.csv"),
    (df_hidrojen_peroktif, "combined_hidrojen.csv"),
    (df_otoklav, "combined_otoklav.csv"),
    (df_kartus, "combined_kartus.csv"),
    (df_reverse, "combined_reverse.csv"),
    (df_yikama, "combined_yikama.csv"),
    (df_oksijen, "combined_oksijen.csv")
]


# Belirtilen dosya yolunu tanımlama
save_path = r"C:\Users\duygu\Desktop\00-GitHub-Repo\00-1INTERNSHIP\INT1_PROJECT\00_Data\02_Birlestirilmis_Kategoriler\final_kategori_csv"

# Dosya yolunu oluşturma (eğer yoksa)
os.makedirs(save_path, exist_ok=True)

# DataFrame'leri CSV olarak kaydetme
file_paths = []  # Kaydedilen dosya yollarını tutmak için
for df, filename in dfs:
    file_path = os.path.join(save_path, filename)
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    file_paths.append(file_path)

file_paths  # Kaydedilen dosya yollarını göstereceğiz

['C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\00_Data\\02_Birlestirilmis_Kategoriler\\final_kategori_csv\\combined_ameliyat.csv',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\00_Data\\02_Birlestirilmis_Kategoriler\\final_kategori_csv\\combined_hidrojen.csv',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\00_Data\\02_Birlestirilmis_Kategoriler\\final_kategori_csv\\combined_otoklav.csv',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\00_Data\\02_Birlestirilmis_Kategoriler\\final_kategori_csv\\combined_kartus.csv',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\00_Data\\02_Birlestirilmis_Kategoriler\\final_kategori_csv\\combined_reverse.csv',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\00_Data\\02_Birlestirilmis_Kategoriler\\final_kategori_csv\\combined_yikama.csv',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INT

In [32]:
# EXCEL FORMATTA SAVE
import os

# DataFrame isimlerini ve dosya isimlerini bir araya getirme
dfs = [
    (df_ameliyat, "combined_ameliyat.xlsx"),
    (df_hidrojen_peroktif, "combined_hidrojen.xlsx"),
    (df_otoklav, "combined_otoklav.xlsx"),
    (df_kartus, "combined_kartus.xlsx"),
    (df_reverse, "combined_reverse.xlsx"),
    (df_yikama, "combined_yikama.xlsx"),
    (df_oksijen, "combined_oksijen.xlsx")
]

# Belirtilen dosya yolunu tanımlama
save_path = r"C:\Users\duygu\Desktop\00-GitHub-Repo\00-1INTERNSHIP\INT1_PROJECT\00_Data\02_Birlestirilmis_Kategoriler\final_kategori_excel"

# Dosya yolunu oluşturma (eğer yoksa)
os.makedirs(save_path, exist_ok=True)

# DataFrame'leri Excel olarak kaydetme
file_paths = []  # Kaydedilen dosya yollarını tutmak için
for df, filename in dfs:
    file_path = os.path.join(save_path, filename)
    df.to_excel(file_path, index=False)
    file_paths.append(file_path)

file_paths  # Kaydedilen dosya yollarını göstereceğiz


['C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\00_Data\\02_Birlestirilmis_Kategoriler\\final_kategori_excel\\combined_ameliyat.xlsx',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\00_Data\\02_Birlestirilmis_Kategoriler\\final_kategori_excel\\combined_hidrojen.xlsx',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\00_Data\\02_Birlestirilmis_Kategoriler\\final_kategori_excel\\combined_otoklav.xlsx',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\00_Data\\02_Birlestirilmis_Kategoriler\\final_kategori_excel\\combined_kartus.xlsx',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\00_Data\\02_Birlestirilmis_Kategoriler\\final_kategori_excel\\combined_reverse.xlsx',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\00_Data\\02_Birlestirilmis_Kategoriler\\final_kategori_excel\\combined_yikama.xlsx',
 'C:\\Users\\duygu\\Desktop\\00-Gi

In [154]:
# Ornek df goster

import pandas as pd

# Dosya yolunu belirtelim
file_path = r"C:\Users\duygu\Desktop\00-GitHub-Repo\00-1INTERNSHIP\00_Data\01_Satis_Uretim\02_Birlestirilmis_Kategoriler\final_kategori_csv\combined_reverse.csv"

# CSV dosyasını okuyalım
try:
    df_read = pd.read_csv(file_path)
    # İlk birkaç satırı göstereceğiz
    df_read.head()
except FileNotFoundError:
    "Dosya yolunda bir hata olabilir veya dosya mevcut olmayabilir."
    
df_read.head()


,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi
0,REVERSE OSMOS,ÖZEL,NaN,NaN,BEZMİALEM VAKIF ÜNİVERSİTESİ,BEZMİALEM VAKIF ÜNİVERSİTESİ,İSTANBUL,PREMİUM 240,NaN,500 LT,2018/08-001,NaN,NaN
1,REVERSE OSMOS,ÖZEL,NaN,NaN,BÜYÜKÇEKMECE DEVLET HASTANESİ,BÜYÜKÇEKMECE DEVLET HASTANESİ,İSTANBUL,PREMİUM 440,NaN,1000 LT,2018/08-002,NaN,NaN
2,REVERSE OSMOS,NaN,NaN,NaN,FOÇA DEVLET HASTANESİ,FOÇA DEVLET HASTANESİ,İZMİR,PREMİUM 240,NaN,500 LT,2018/08-003,NaN,NaN
3,REVERSE OSMOS,NaN,NaN,NaN,Gemlik Devlet Hastanesi,Gemlik Devlet Hastanesi,bursa,PREMİUM 440,NaN,1000 LT,2018/08-004,NaN,NaN
4,REVERSE OSMOS,YURTDIŞI,2018/186,21.06.2018,SİMAD ÜNİVERSİTESİ,SİMAD ÜNİVERSİTESİ,SOMALİ,PREMİUM 140,NaN,250 LT,2018/08-005,NaN,NaN


# **Tüm Kategorileri CONCAT Etme** 

## DF'leri son kontrol

In [164]:
def create_columns_table(**dfs):
    """
    Verilen DataFrame'lerin sütun isimlerini tablo halinde gösterir.
    DataFrame isimlerini başlık olarak kullanır.
    
    :param dfs: Anahtar-değer çiftleri olarak DataFrame'ler (örneğin, df_ameliyat=df1)
    :return: Sütun isimlerini gösteren bir DataFrame
    """
    # Sütun isimlerini bir listede topla
    column_lists = [df.columns.tolist() for df in dfs.values()]
    
    # En uzun sütun listesine göre eksik sütunları doldurmak için her listeyi aynı uzunlukta yap
    max_len = max(len(cols) for cols in column_lists)
    column_lists = [cols + [None] * (max_len - len(cols)) for cols in column_lists]
    
    # DataFrame isimlerini başlık olarak kullan
    col_table = pd.DataFrame({name: cols for name, cols in zip(dfs.keys(), column_lists)})
    return col_table

In [166]:
create_columns_table(
    df_ameliyat=df_ameliyat, 
    df_otoklav=df_otoklav,
    df_hidrojen_peroktif=df_hidrojen_peroktif,
    df_kartus=df_kartus,
    df_reverse=df_reverse,
    df_yikama=df_yikama,
    df_oksijen=df_oksijen
)

,df_ameliyat,df_otoklav,df_hidrojen_peroktif,df_kartus,df_reverse,df_yikama,df_oksijen
0,kategori,kategori,kategori,kategori,kategori,kategori,kategori
1,satis_turu,satis_turu,satis_turu,satis_turu,satis_turu,satis_turu,satis_turu
2,siparis_no,siparis_no,siparis_no,siparis_no,siparis_no,siparis_no,siparis_no
3,siparis_tarihi,siparis_tarihi,siparis_tarihi,siparis_tarihi,siparis_tarihi,siparis_tarihi,siparis_tarihi
4,siparisi_veren,siparisi_veren,siparisi_veren,siparisi_veren,siparisi_veren,siparisi_veren,siparisi_veren
5,kurulum_yeri,kurulum_yeri,kurulum_yeri,kurulum_yeri,kurulum_yeri,kurulum_yeri,kurulum_yeri
6,il,il,il,il,il,il,il
7,urun_modeli,urun_modeli,urun_modeli,urun_modeli,urun_modeli,urun_modeli,urun_modeli
8,urun_adi,urun_adi,urun_adi,urun_adi,urun_adi,urun_adi,urun_adi
9,litre,litre,litre,litre,litre,litre,litre


### df samples

In [25]:
df_ameliyat.sample(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi
382,AMELIYAT MASASI,2024 YILINA AKTARILDI,2023/393,2023-11-07,2024 YILANA AKTARILDI,2024 YILANA AKTARILDI,2024 YILANA AKTARILDI,GOLDBERG 5 S,AMELİYAT MASASI,NaN,NaN,NaN,NaN,NaN,NaN
304,AMELIYAT MASASI,YURTDIŞI,2022/321,2022-08-17,FAS,FAS,FAS,GOLDBERG 5S,AMELİYAT MASASI,NaN,2022/02-030,22,2022-09-08,NaN,NaN


In [28]:
df_otoklav.sample(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi,uts_no
38,OKSIJEN,YURTDIŞI,2021/339,2021-09-16,UKRAYNA,UKRAYNA,UKRAYNA,OXY-GOLD 37,NaN,NaN,2021/06-034,2021-10-11 00:00:00,25.0,NaN,NaN,NaN
19,OKSIJEN,YURTDIŞI,2021/254,2021-07-26,ESTONYA,ESTONYA,ESTONYA,OXY-GOLD 37,NaN,NaN,2021/06-012,2021-08-06 00:00:00,NaN,NaN,NaN,NaN


In [31]:
df_hidrojen_peroktif.sample(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi
11,HIDROJEN PEROKSIT,DMO-YÜKSELİŞ,2018/171,NaN,DMO İZMİR BÖLGE MÜDÜRLÜĞÜ,MERKEZ EFENDİ DEVLET HASTANESİ,manisa,TKF 120,120 LT ÇİFT KAPI PLAZMA CİHAZI,120 LT,2018/04-004,62 GÜN,NaN,NaN,NaN
85,HIDROJEN PEROKSIT,DMO,2022/079,2022-04-04,DMO ELAZIĞ BÖLGE MÜDÜRLÜĞÜ,MALATYA İL SAĞLIK MÜDÜRLÜĞÜ,MALATYA,GOLBERG GP 120,120 LT TEK KAPILI PLAZMA CİHAZI,120 LT,2022/04-004,60,2022-06-03,NaN,NaN


In [32]:
df_kartus.sample(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,aciklama,solusyon_fatura,bidon_acilisi
25237,KARTUS,ÖZEL,2024/57,2024-05-13 00:00:00,KARS HARAKANİ DEVLET HASTANESİ,NaN,KARS,NaN,KARTUŞ,NaN,2024/07-2107,NaN,NaN,NaN,NaN,NaN
1532,KARTUS,NaN,2018/083,2018-10-27,ÇOBANBEY-ELBAB HASTANESİ,NaN,SURİYE,NaN,KARTUŞ,NaN,2018/07-1524,NaN,NaN,SATIŞ,NaN,NaN


In [33]:
df_reverse.sample(2)

,"kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi"
118,"REVERSE OSMOS ,ÖZEL,2022/300,28.06.2022,STN MA..."
107,"REVERSE OSMOS ,YURTDIŞI,2022/203,18.05.2022,SM..."


In [34]:
df_yikama.sample(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi
372,YIKAMA,ÖZEL,2024/198,2024-04-05,ANKARA 75. YIL ADSM,ANKARA 75. YIL ADSM,ANKARA,GY 150,150 LT TEZGAHALTI YIKAMA DEZENFEKSİYON KURUTMA...,150 LT,2024/05-036,65 GÜN,2024-06-04,NaN,NaN
124,YIKAMA,DMO,2021/139,2021-04-08,DMO İZMİR BÖLGE MÜDÜRLÜĞÜ,SINDIRGI DEVLET HASTANESİ,BALIKESİR,TSY 290 O,290 LT ÇİFT KAPILI YIKAMA DEZENFEKSİYON CİHAZI,290 LT,2021/05-014,64.0,2021-06-11 00:00:00,NaN,NaN


In [35]:
df_oksijen.sample(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi,uts_no
56,OKSIJEN,YURTDIŞI,2021/458,2021-12-18,UKRAYNA,UKRAYNA,UKRAYNA,OXY-GOLD 45,NaN,NaN,2021/06-052,2021-12-30 00:00:00,12.0,NaN,NaN,NaN
33,OKSIJEN,YURTDIŞI,2021/301,2021-08-26,CEZAYİR,CEZAYİR,CEZAYİR,OXY-GOLD 45,NaN,NaN,2021/06-029,2021-08-31 00:00:00,NaN,NaN,NaN,NaN


In [37]:
# df_ameliyat 
# df_otoklav
# df_hidrojen_peroktif
# df_kartus
# df_reverse
# df_yikama
# df_oksijen

# DataFrame'leri alt alta birleştir
amel_otok = pd.concat([df_ameliyat, df_otoklav], ignore_index=True, sort=False)
amel_otok.sample(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi
273,AMELIYAT MASASI,ÖZEL,2022/130,2022-04-07,ER TIBBİ CİHAZLAR,ORDU EAH,ORDU,GOLDBERG 5S,AMELİYAT MASASI,NaN,2022/02-003,30,2022-05-07,NaN,NaN
371,AMELIYAT MASASI,DMO,2023/340,2023-12-06,DMO 1 NOLU OPERASYONEL SÜREÇLER DAİRE BAŞKANLIĞI,SB KAMU HASTANELERİ GENEL MÜDÜRLÜĞÜ (DEPREM BÖ...,HATAY DEPREM BÖLGESİ,GOLDBERG 5 S,AMELİYAT MASASI,NaN,2023/02-054,70 GÜN,2024-02-14,NaN,NaN


In [38]:
# df_ameliyat 
# df_otoklav
# df_hidrojen_peroktif
# df_kartus
# df_reverse
# df_yikama
# df_oksijen

# DataFrame'leri alt alta birleştir
amel_otok_hid = pd.concat([df_ameliyat, df_otoklav,df_hidrojen_peroktif], ignore_index=True, sort=False)
amel_otok_hid.sample(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi
171,AMELIYAT MASASI,DMO,2020/180,2020-04-29,DMO GENEL MÜDÜRLÜĞÜ,ŞIRNAK KADIN DOĞUM VE ÇOCUK HASTALIKLARI HASTA...,ŞIRNAK,GOLDBERG 5S,AMELİYAT MASASI,*,2020/02-042,2020-05-10 00:00:00,11,NaN,
1591,OTOKLAV,YURTDIŞI,2023/406,2024-01-10,FAS,FAS,FAS,GOLDBERG 550 D,550 LT ÇİFT KAPILI OTOKLAV CİHAZI,550 LT,2024/01-010,20 GÜN,2024-02-20,NaN,NaN


In [52]:
# df_ameliyat 
# df_otoklav
# df_hidrojen_peroktif
# df_kartus
# df_reverse
# df_yikama
# df_oksijen

# DataFrame'leri alt alta birleştir
amel_otok_hid_kart = pd.concat([df_ameliyat, df_otoklav,df_hidrojen_peroktif,df_kartus], ignore_index=True, sort=False)
amel_otok_hid_kart.sample(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi,aciklama,solusyon_fatura,bidon_acilisi
122,AMELIYAT MASASI,DEMO,2020/036,2020-03-09,ORDU KADIN DOĞUM HASTANESİ,ORDU KADIN DOĞUM HASTANESİ,ORDU,GOLDBERG 5S,AMELİYAT MASASI,*,2020/02-004,2020-03-09 00:00:00,15,NaN,,NaN,NaN,NaN
9040,KARTUS,ÖZEL,2021/002,2021-01-07,DTH SAĞLIK-ONKOLOJİ HASTANESİ,NaN,ANKARA,NaN,KARTUŞ,NaN,2021/07-019,NaN,NaN,NaN,NaN,SATIŞ BÜYÜK KARTUŞ,NaN,NaN


In [57]:
# df_ameliyat 
# df_otoklav
# df_hidrojen_peroktif
# df_kartus
# df_reverse
# df_yikama
# df_oksijen

# DataFrame'leri alt alta birleştir
amel_otok_hid_kart_rev = pd.concat([df_ameliyat, df_otoklav,df_hidrojen_peroktif,df_kartus,df_reverse], ignore_index=True, sort=False)
amel_otok_hid_kart_rev.sample(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi,aciklama,solusyon_fatura,bidon_acilisi,"kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi"
12004,KARTUS,ÖZEL,2021/098,2021-09-10,ALYANS MEDİKAL,NaN,ANKARA,NaN,KARTUŞ,NaN,2021/07-2983,NaN,NaN,NaN,NaN,KAZIM ABİ KARTUŞ,NaN,NaN,NaN
573,OTOKLAV,YURTDIŞI,2018/273,2018-09-19,ETİYOPYA,ETİYOPYA,ETİYOPYA,STR 75 V,75 LT VAKUMLU OTOKLAV CİHAZI,75 LT,2018/01-100,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
# df_ameliyat 
# df_otoklav
# df_hidrojen_peroktif
# df_kartus
# df_reverse
# df_yikama
# df_oksijen

# DataFrame'leri alt alta birleştir
amel_otok_hid_kart_rev_yik = pd.concat([df_ameliyat, df_otoklav,df_hidrojen_peroktif,df_kartus,df_reverse,df_yikama], ignore_index=True, sort=False)
amel_otok_hid_kart_rev_yik.sample(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi,aciklama,solusyon_fatura,bidon_acilisi,"kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi"
693,OTOKLAV,DMO,2019/255,2019-11-25,DMO TRABZON BÖLGE MÜDÜRLÜĞÜ,VAKFIKEBİR DEVLET HASTANESİ,TRABZON,ERS 6613 D,550 LT ÇİFT KAYAR KAPILI OTOKLAV CİHAZI,550 LT,2019/01-085,59,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9520,KARTUS,SATIŞ,2021/025,2021-02-23,ARNAVUTLUK FİER HASTANESİ,NaN,ARNAVUTLUK,NaN,KARTUŞ,NaN,2021/07-499,NaN,NaN,NaN,NaN,CİHAZIN YANINDA VERİLEN,NaN,NaN,NaN


In [126]:
# df_ameliyat 
# df_otoklav
# df_hidrojen_peroktif
# df_kartus
# df_reverse
# df_yikama
# df_oksijen

# DataFrame'leri alt alta birleştir
amel_otok_hid_kart_rev_yik_okj = pd.concat([df_ameliyat, df_otoklav,df_hidrojen_peroktif,df_kartus,df_reverse,df_yikama,df_oksijen], ignore_index=True, sort=False)
amel_otok_hid_kart_rev_yik_okj.sample(2)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi,aciklama,solusyon_fatura,bidon_acilisi,"kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi",uts_no
451,AMELIYAT MASASI,YURTDIŞI,2023/390,2024-02-14,IP,PAKİSTAN,PAKİSTAN,GOLDBERG 5 S,AMELİYAT MASASI,NaN,2024/02-008,60 GÜN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13586,KARTUS,YURTDIŞI,2022/001,2022-01-05,ÖZBEKİSTAN,NaN,ÖZBEKİSTAN,NaN,KARTUŞ,NaN,2022/07-048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Concat

In [170]:
# Tüm DataFrame'leri alt alta birleştiriyoruz
dfs_list = [df_ameliyat, 
            df_otoklav,
            df_hidrojen_peroktif,
            df_kartus,
            df_reverse,
            df_yikama,
            df_oksijen,
           ]

concat_df = pd.concat(dfs_list, ignore_index=True, sort=False)

In [182]:
# Sonuç DataFrame'i görüntülemek için
print(concat_df.shape)
print(concat_df.columns)
concat_df.sample(3)

(32646, 19)
Index(['kategori', 'satis_turu', 'siparis_no', 'siparis_tarihi', 'siparisi_veren', 'kurulum_yeri', 'il', 'urun_modeli', 'urun_adi', 'litre', 'seri_no', 'teslim_suresi', 'teslim_tarihi', 'siparis_durumu', 'irsaliye_tarihi', 'aciklama', 'solusyon_fatura', 'bidon_acilisi', 'uts_no'], dtype='object')


,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi,aciklama,solusyon_fatura,bidon_acilisi,uts_no
25917,KARTUS,ÖZEL,2024/26,2024-02-22 00:00:00,ÖZEL A HASTANESİ,NaN,İSTANBUL,NaN,KARTUŞ,NaN,2024/07-0852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10624,KARTUS,DOĞRUDAN TEMİN,2021/068,2021-05-28,BAKIRKÖY DR.SADİ KONUK EĞİTİM VE ARAŞTIRMA HAS...,NaN,İSTANBUL,NaN,KARTUŞ,NaN,2021/07-1603,NaN,NaN,NaN,NaN,SATIŞ BÜYÜK KARTUŞ,NaN,NaN,NaN
1614,OTOKLAV,YURTDIŞI,2024/037,2024-03-14,UN,MOLDOVA,MOLDOVA,GOLDBERG 422 D,422 LT ÇİFT KAPILI OTOKLAV CİHAZI,422 LT,2024/01-063,40 GÜN,2024-04-15,NaN,NaN,NaN,NaN,NaN,NaN


## FINAL SUTUN LISTESI

In [ ]:
Index(['kategori', 
       'satis_turu', 
       'siparis_no', 
       'siparis_tarihi', 
       'siparisi_veren', 
       'kurulum_yeri', 
       'il', 
       'urun_modeli', 
       'urun_adi', 
       'litre', 
       'seri_no', 
       'teslim_suresi', 
       'teslim_tarihi', 
       'siparis_durumu', 
       'irsaliye_tarihi', 
       'aciklama', 
       'solusyon_fatura', 
       'bidon_acilisi', 
       'uts_no'
      ], dtype='object')

In [ ]:
# # Move Colunms index Fonksiyonu cagir

# # combined_df üzerinde 'urun_adi' 'urun_modeli' nden sonra getirmek icin
# concat_df = move_column(combined_df, 'urun_adi', 8)

# concat_df.head()


## Save Final CSV 

In [ ]:
concat_df.to_csv('merged_satis_uretim_data_27_Ekim.csv', index=False, encoding='utf-8-sig')

In [3]:
import pandas as pd 

df_merged= pd.read_csv('merged_satis_uretim_data_27_Ekim.csv')
df_merged.sample(5)

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi,aciklama,solusyon_fatura,bidon_acilisi,uts_no
3688,KARTUS,ÖZEL,2019/005,2019-01-16,BURSA YÜKSEK İHTİSAS EĞİTİM VE ARAŞTIRMA HASTA...,NaN,BURSA,NaN,KARTUŞ,NaN,2019/07-0034,NaN,NaN,NaN,NaN,DEĞİŞİM,19.4.2018-HPS2018-00311,2019-01-02,NaN
2127,KARTUS,NaN,2018/015,2018-02-06,FATİH SULTAN MEHMET EĞİTİM VE ARAŞTIRMA HASTA...,NaN,İSTANBUL,NaN,KARTUŞ,NaN,2018/07-276,NaN,NaN,NaN,NaN,SATIŞ,NaN,NaN,NaN
9738,KARTUS,DOĞRUDAN TEMİN,2021/032,2021-03-10,FİNİKE DEVLET HASTANESİ,NaN,ANTALYA,NaN,KARTUŞ,NaN,2021/07-717,NaN,NaN,NaN,NaN,BÜYÜK KARTUŞ SATIŞ,NaN,NaN,NaN
9217,KARTUS,YURTDIŞI,2021/010,2021-06-30,İPTAL VİETNAM KARTUŞU,NaN,VİETNAM,NaN,KARTUŞ,NaN,2021/07-196,NaN,NaN,NaN,NaN,İPTAL,NaN,NaN,NaN
12843,KARTUS,ÖZEL,2021/121,2021-11-10,MEDİKAL PARK,NaN,İSTANBUL,NaN,KARTUŞ,NaN,2021/07-3822,NaN,NaN,NaN,NaN,ÖZEL,NaN,NaN,NaN


In [4]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32646 entries, 0 to 32645
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   kategori         32646 non-null  object 
 1   satis_turu       29480 non-null  object 
 2   siparis_no       27743 non-null  object 
 3   siparis_tarihi   30830 non-null  object 
 4   siparisi_veren   31216 non-null  object 
 5   kurulum_yeri     2519 non-null   object 
 6   il               31083 non-null  object 
 7   urun_modeli      2450 non-null   object 
 8   urun_adi         31119 non-null  object 
 9   litre            2245 non-null   object 
 10  seri_no          32443 non-null  object 
 11  teslim_suresi    2225 non-null   object 
 12  teslim_tarihi    2144 non-null   object 
 13  siparis_durumu   0 non-null      float64
 14  irsaliye_tarihi  90 non-null     object 
 15  aciklama         11627 non-null  object 
 16  solusyon_fatura  5369 non-null   object 
 17  bidon_acilis

In [198]:
missing_values(df_merged)

,count,percentage
kategori,0,0.00%
satis_turu,3166,9.70%
siparis_no,4903,15.02%
siparis_tarihi,1816,5.56%
siparisi_veren,1430,4.38%
kurulum_yeri,30127,92.28%
il,1563,4.79%
urun_modeli,30196,92.50%
urun_adi,1527,4.68%
litre,30401,93.12%


In [201]:
# Kategorilere göre her sütundaki dolu değerlerin sayısını hesaplayan DataFrame
category_counts_df = df_merged.groupby('kategori').apply(lambda x: x.notnull().sum())

# Sonucu döndürelim (görüntülemek için bu kodu kullanabilirsiniz)
category_counts_df.T


kategori,AMELIYAT MASASI,HIDROJEN PEROKSIT,KARTUS,OKSIJEN,OTOKLAV,REVERSE OSMOS,YIKAMA
kategori,475,135,30126,71,1242,204,393
satis_turu,475,135,26963,71,1242,201,393
siparis_no,475,134,25234,70,1242,195,393
siparis_tarihi,475,134,28317,70,1242,199,393
siparisi_veren,475,135,28697,70,1242,204,393
kurulum_yeri,475,135,0,70,1242,204,393
il,473,134,28585,70,1228,202,391
urun_modeli,475,135,0,70,1181,204,385
urun_adi,475,135,28794,10,1242,70,393
litre,271,135,0,2,1242,202,393


In [5]:
# Drop siparis_durumu: hicbir yilda data yok tamamen bos, sadece sutun ismi cikiyor, bu nedenle drop 
df_merged = df_merged.drop('siparis_durumu', axis=1)

df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32646 entries, 0 to 32645
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   kategori         32646 non-null  object 
 1   satis_turu       29480 non-null  object 
 2   siparis_no       27743 non-null  object 
 3   siparis_tarihi   30830 non-null  object 
 4   siparisi_veren   31216 non-null  object 
 5   kurulum_yeri     2519 non-null   object 
 6   il               31083 non-null  object 
 7   urun_modeli      2450 non-null   object 
 8   urun_adi         31119 non-null  object 
 9   litre            2245 non-null   object 
 10  seri_no          32443 non-null  object 
 11  teslim_suresi    2225 non-null   object 
 12  teslim_tarihi    2144 non-null   object 
 13  irsaliye_tarihi  90 non-null     object 
 14  aciklama         11627 non-null  object 
 15  solusyon_fatura  5369 non-null   object 
 16  bidon_acilisi    5369 non-null   object 
 17  uts_no      

In [6]:
df_merged.to_csv('merged_satis_uretim_data1_27_Ekim.csv', index=False, encoding='utf-8-sig')

# df without KARTUS

In [210]:
# 'KARTUS' kategorisini hariç tutarak yeni bir DataFrame oluşturma
df_filtered = df_merged[df_merged['kategori'] != 'KARTUS']
df_filtered.head(2)


,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,irsaliye_tarihi,aciklama,solusyon_fatura,bidon_acilisi,uts_no
0,AMELIYAT MASASI,ÖZEL,2018/006,2018-06-29,ÖZEL FSM TIP MERKEZİ,ÖZEL FSM TIP MERKEZİ,İSTANBUL,GOLDBERG 5S,AMELİYAT MASASI,*,2018/02-012,20 GÜN,2018-07-18,NaN,NaN,NaN,NaN,NaN
1,AMELIYAT MASASI,DEMO,2018/007,2018-02-14,DEMO,DERİNCE EĞİTİM VE ARAŞTIRMA HASTANESİ,KOCAELİ,GOLDBERG 5S,AMELİYAT MASASI,*,2018/02-003,21 GÜN,NaN,NaN,NaN,NaN,NaN,NaN


In [211]:
# Kategorilere göre her sütundaki dolu değerlerin sayısını hesaplayan DataFrame
category_counts_df_filtered = df_filtered.groupby('kategori').apply(lambda x: x.notnull().sum())

# Sonucu görüntülemek için
category_counts_df_filtered

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,irsaliye_tarihi,aciklama,solusyon_fatura,bidon_acilisi,uts_no
kategori,,,,,,,,,,,,,,,,,,
AMELIYAT MASASI,475,475,475,475,475,475,473,475,475,271,438,425,406,82,0,0,0,0
HIDROJEN PEROKSIT,135,135,134,134,135,135,134,135,135,135,134,135,126,2,0,0,0,0
OKSIJEN,71,71,70,70,70,70,70,70,10,2,70,43,65,0,0,0,0,3
OTOKLAV,1242,1242,1242,1242,1242,1242,1228,1181,1242,1242,1176,1047,982,6,0,0,0,0
REVERSE OSMOS,204,201,195,199,204,204,202,204,70,202,204,192,190,0,0,0,0,0
YIKAMA,393,393,393,393,393,393,391,385,393,393,383,383,372,0,0,0,0,0


In [212]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2520 entries, 0 to 32645
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   kategori         2520 non-null   object 
 1   satis_turu       2517 non-null   object 
 2   siparis_no       2509 non-null   object 
 3   siparis_tarihi   2513 non-null   object 
 4   siparisi_veren   2519 non-null   object 
 5   kurulum_yeri     2519 non-null   object 
 6   il               2498 non-null   object 
 7   urun_modeli      2450 non-null   object 
 8   urun_adi         2325 non-null   object 
 9   litre            2245 non-null   object 
 10  seri_no          2405 non-null   object 
 11  teslim_suresi    2225 non-null   object 
 12  teslim_tarihi    2141 non-null   object 
 13  irsaliye_tarihi  90 non-null     object 
 14  aciklama         0 non-null      object 
 15  solusyon_fatura  0 non-null      object 
 16  bidon_acilisi    0 non-null      object 
 17  uts_no           3

In [213]:
#NELER YAPILDI:

# Tum kategorilere gore olusturulan csv dosyalari incelendi, gerekli yerlerde yazim ve sutun isimleri duzenlendi
# Tum kategoriler de yek bir df olarak CONCAT edilerek birlestirildi. 
# KARTUS datasinda bir cok degerin olmadigi goruldu
# `siparis_durumu` sutunu'unda hic data yok ham data setleri tek tek kontrol edildi, tamamen bos, drop edildi. 
# Son duzenlenmis hali ile dataset `merged_satis_uretim_data_27_Ekim` adinda kayit edildi. 

# NOTLAR:
# Kartus'ta 30bine yakin data var ama ortak sutunlarda degerler yok (kurulum_yeri gibi 0) 
# bu nedenle bazi sutunlarda missing deger yuzdesi cok goruluyor. Bunu dikkate alarak analizleri yapalim. 
# Hicbir datayi drop etmeyecegiz, ignore ederek analizleri yapabiliriz. 